In [ ]:
import os
import numpy as np
from astropy.io import fits
from astropy.nddata import CCDData
import ccdproc
from astropy import units as u
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
from collections import defaultdict

# Base route
base_path = "./"

# Function to get valid FITS files
def get_fits_files(folder_path, prefix):
    files = []
    for f in os.listdir(folder_path):
        full_path = os.path.join(folder_path, f)
        if f.startswith(prefix) and os.path.isfile(full_path) and f.lower().endswith(('.fits', '.fit')):
            files.append(full_path)
    return sorted(files)

# File paths - now checking that they are valid files
bias_files = get_fits_files(base_path, "bias")
dark_files = get_fits_files(base_path, "dark")
flat_files = get_fits_files(base_path, "flat")
light_files = get_fits_files(base_path, "M83")

# Verify that there are files to process
if not bias_files:
    raise ValueError("No valid bias files were found in the specified path")
if not dark_files:
    print("Warning: No valid dark files found")
if not flat_files:
    raise ValueError("No valid flat files were found")
if not light_files:
    raise ValueError("No valid light files were found")

# Create output folders
aligned_dir = os.path.join(base_path, "Aligned")
processed_dir = os.path.join(base_path, "Processed")
summed_dir = os.path.join(base_path, "Summed")

os.makedirs(aligned_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(summed_dir, exist_ok=True)

def combine_images(file_list):
    data_stack = []
    for f in file_list:
        try:
            data = fits.getdata(f).astype(float)
            data_stack.append(data)
        except Exception as e:
            print(f"Error al procesar {f}: {str(e)}")
            continue
    
    if not data_stack:
        raise ValueError("Could not load images to combine")
    
    return np.median(data_stack, axis=0)

# Process master bias
try:
    master_bias = combine_images(bias_files)
    fits.writeto(os.path.join(base_path, "master_bias.fits"), master_bias, overwrite=True)
except Exception as e:
    raise ValueError(f"Error al crear master bias: {str(e)}")

# Process master dark (if there are files)
master_dark = np.zeros_like(master_bias) # Default value if there are no darks
if dark_files:
    try:
        master_dark = combine_images(dark_files)
        fits.writeto(os.path.join(base_path, "master_dark.fits"), master_dark, overwrite=True)
    except Exception as e:
        print(f"Warning Error creating master dark: {str(e)}. Using array of zeros.")

def extract_filter(filename):
    filename_lower = filename.lower()
    if "_g_" in filename_lower:
        return "g"
    elif "_r_" in filename_lower:
        return "r"
    elif "_i_" in filename_lower:
        return "i"
    return None

master_flats = {}

for filt in ["g", "r", "i"]:
    flat_filt_files = [f for f in flat_files if f"_{filt}_" in f.lower()]
    if not flat_filt_files:
        print(f"Warning: No flats found for the filter {filt}")
        continue
    
    try:
        flat_stack = [(fits.getdata(f).astype(float) - master_bias) for f in flat_filt_files]
        median_flat = np.median(flat_stack, axis=0)
        norm_flat = median_flat / np.mean(median_flat)
        master_flats[filt] = norm_flat
        fits.writeto(os.path.join(base_path, f"master_flat_{filt}.fits"), norm_flat, overwrite=True)
    except Exception as e:
        print(f"Error processing flats for filter {filt}: {str(e)}")
        continue

def reduce_light(path, filt):
    try:
        data = fits.getdata(path).astype(float)
        if filt not in master_flats:
            raise ValueError(f"There is no master flat for the filter {filt}")
        reduced = (data - master_dark) / master_flats[filt]
        return reduced
    except Exception as e:
        print(f"Error reducing {path}: {str(e)}")
        return None

def enhanced_align_images(image_list, reference_image=None, upsample_factor=10):
    if not image_list:
        return []
    
    aligned = []
    
    if reference_image is None:
        ref_image = image_list[0]
        aligned.append(ref_image)
        images_to_align = image_list[1:]
    else:
        ref_image = reference_image
        images_to_align = image_list
    
    #Preprocessing: crop edges to avoid artifacts
    crop_pixels = 50
    h, w = ref_image.shape
    ref_cropped = ref_image[crop_pixels:h-crop_pixels, crop_pixels:w-crop_pixels]
    
    for img in images_to_align:
        try:
            img_cropped = img[crop_pixels:h-crop_pixels, crop_pixels:w-crop_pixels]
            
            # Use phase_cross_correlation more accurately
            shift_val, error, diffphase = phase_cross_correlation(
                ref_cropped, 
                img_cropped, 
                upsample_factor=upsample_factor,
                normalization=None
            )
            
           # Apply the shift to the entire image
            shifted = shift(img, shift_val, mode='reflect')
            aligned.append(shifted)
            
            print(f"Calculated displacement: {shift_val} (error: {error:.4f}, diffphase: {diffphase:.4f})")
        except Exception as e:
            print(f"Error aligning image: {str(e)}")
            continue
    
    return aligned if reference_image is None else aligned

light_by_filter = defaultdict(list)
for f in light_files:
    filt = extract_filter(f)
    if filt:
        light_by_filter[filt].append(f)

# Process filters that align well first (r and i)
reference_image = None
for filt in ["r", "i"]:
    if filt in light_by_filter and light_by_filter[filt]:
        print(f"\nProcessing filter {filt} with {len(light_by_filter[filt])} imagen...")
        
        reduced_imgs = []
        for f in light_by_filter[filt]:
            reduced = reduce_light(f, filt)
            if reduced is not None:
                reduced_imgs.append(reduced)
        
        if reduced_imgs:
            aligned_imgs = enhanced_align_images(reduced_imgs, upsample_factor=20)
            
            for i, img in enumerate(aligned_imgs):
                output_path = os.path.join(aligned_dir, f"aligned_{filt}_{i+1:03d}.fits")
                fits.writeto(output_path, img, overwrite=True)
                print(f"Save: {output_path}")
            
            # Save the first aligned image as a reference for the g filter
            if reference_image is None and aligned_imgs:
                reference_image = aligned_imgs[0]

# Process the g filter using the reference of r or i
if "g" in light_by_filter and light_by_filter["g"]:
    print("\nProcessing g filter using r/i reference...")
    
    reduced_imgs = []
    for f in light_by_filter["g"]:
        reduced = reduce_light(f, "g")
        if reduced is not None:
            reduced_imgs.append(reduced)
    
    if reduced_imgs:
        if reference_image is not None:
            print("Aligning with reference image")
            aligned_imgs = enhanced_align_images(reduced_imgs, reference_image=reference_image, upsample_factor=20)
        else:
            print("No reference image available, aligning g internally")
            aligned_imgs = enhanced_align_images(reduced_imgs, upsample_factor=20)
        
        for i, img in enumerate(aligned_imgs):
            output_path = os.path.join(aligned_dir, f"aligned_g_{i+1:03d}.fits")
            fits.writeto(output_path, img, overwrite=True)
            print(f"Guardado: {output_path}")

# Combine aligned images
for filt in ["g", "r", "i"]:
    if filt in light_by_filter:
        aligned_files = []
        for f in os.listdir(aligned_dir):
            if f.startswith(f"aligned_{filt}") and f.lower().endswith(('.fits', '.fit')):
                aligned_files.append(os.path.join(aligned_dir, f))
        
        aligned_files = sorted(aligned_files)
        if aligned_files:
            stack = []
            for f in aligned_files:
                try:
                    data = fits.getdata(f).astype(float)
                    stack.append(data)
                except Exception as e:
                    print(f"Error {f}: {str(e)}")
                    continue
            
            if stack:
                summed = np.median(stack, axis=0)
                summed_path = os.path.join(summed_dir, f"stacked_{filt}.fits")
                fits.writeto(summed_path, summed, overwrite=True)
                print(f"Save: {summed_path}")
            else:
                print(f"There are no valid images to combine in the filter. {filt}")
        else:
            print(f"No aligned files found for the filter {filt}")

print("\nProcess completed!")

In [ ]:
import os
import numpy as np
from astropy.io import fits
from image_registration import chi2_shift
from image_registration.fft_tools import shift as img_shift

# Base path
base_path = "./"
summed_dir = os.path.join(base_path, "Summed")
aligned_summed_dir = os.path.join(base_path, "Aligned_Summed")

# Create output folder if it doesn't exist
os.makedirs(aligned_summed_dir, exist_ok=True)

# Load the stacked images
def load_stacked_images():
    stacked_files = {
        'g': os.path.join(summed_dir, "stacked_g.fits"),
        'r': os.path.join(summed_dir, "stacked_r.fits"),
        'i': os.path.join(summed_dir, "stacked_i.fits")
    }
    
    images = {}
    for filt, path in stacked_files.items():
        if os.path.exists(path):
            images[filt] = fits.getdata(path)
        else:
            raise FileNotFoundError(f"File not found: {path}")
    
    return images

# Align images using chi2_shift
def align_with_chi2(images):
    # Choose the image with best quality as reference (typically r)
    reference = images['r']
    aligned = {'r': reference}
    
    # Parameters for chi2_shift
    noise_level = np.std(reference)  # Estimate of noise level
    upsample_factor = 'auto'  # You can adjust this if needed
    
    print("\nStarting alignment with chi2_shift...")
    
    # Align g with respect to r
    print("Aligning filter g...")
    xoff_g, yoff_g, exoff_g, eyoff_g = chi2_shift(
        reference, images['g'], noise_level,
        return_error=True, upsample_factor=upsample_factor
    )
    aligned['g'] = img_shift.shiftnd(images['g'], (-yoff_g, -xoff_g))
    print(f"Displacement g: x={xoff_g:.2f} ± {exoff_g:.2f}, y={yoff_g:.2f} ± {eyoff_g:.2f}")
    
    # Align i with respect to r
    print("Aligning filter i...")
    xoff_i, yoff_i, exoff_i, eyoff_i = chi2_shift(
        reference, images['i'], noise_level,
        return_error=True, upsample_factor=upsample_factor
    )
    aligned['i'] = img_shift.shiftnd(images['i'], (-yoff_i, -xoff_i))
    print(f"Displacement i: x={xoff_i:.2f} ± {exoff_i:.2f}, y={yoff_i:.2f} ± {eyoff_i:.2f}")
    
    return aligned

# Save aligned images
def save_aligned_images(aligned_images):
    for filt, img in aligned_images.items():
        output_path = os.path.join(aligned_summed_dir, f"aligned_stacked_{filt}.fits")
        fits.writeto(output_path, img, overwrite=True)
        print(f"Aligned image {filt} saved to: {output_path}")

# Main process
def main():
    try:
        # Load images
        print("Loading stacked images...")
        images = load_stacked_images()
        
        # Align images
        aligned_images = align_with_chi2(images)
        
        # Save results
        save_aligned_images(aligned_images)
        
        print("\nAlignment completed successfully!")
    except Exception as e:
        print(f"\nError during the process: {str(e)}")

if __name__ == "__main__":
    main()
